# Assistant testing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pprint

from npc.apis.gmail_client import GmailClient, GmailSearchOptions, GmailThread
from npc.apis.llm_client import Model
from npc.prompts.ai_assistant.email_action_template import EmailDestination
from npc.simulators.ai_assistant.tools.email_briefing_generator import EmailBriefingGenerator
from npc.simulators.ai_assistant.tools.email_summarizer import EmailSummarizer
from npc.simulators.ai_assistant.tools.inbox_manager import InboxManager, InboxManagerShell

In [ ]:
gmail_client = GmailClient()

max_threads = 5
unread_threads = gmail_client.retrieve_threads(
    GmailSearchOptions(unread=True),
    max_results=max_threads,
)

def email_str(thread: GmailThread):
    email = gmail_client.get_latest_email(thread)
    return f"{email.subject}\nFrom: {email.sender}"
print("\n\n".join([email_str(email) for email in unread_threads]))

In [ ]:
email_summarizer = EmailSummarizer(Model.HAIKU)
for i in range(max_threads):
    email = gmail_client.get_latest_email(unread_threads[i])
    pprint.pprint(email_summarizer.summarize(email), width=140)

In [ ]:
email_briefing_generator = EmailBriefingGenerator(Model.SONNET, email_summarizer)
# email_briefing_generator = EmailBriefingGenerator(Model.HAIKU, email_summarizer)
briefing = email_briefing_generator.generate_briefing(unread_threads)

from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
console = Console()
console.print(Panel(
    Markdown(briefing),
    title="[bold cyan]Email Briefing[/bold cyan]",
    border_style="bright_blue"
))

In [ ]:
label_ids = {
    "CATEGORY_PERSONAL": "CATEGORY_PERSONAL",
    "CATEGORY_PROMOTIONS": "CATEGORY_PROMOTIONS",
    "CATEGORY_SOCIAL": "CATEGORY_SOCIAL",
    "CATEGORY_UPDATES": "CATEGORY_UPDATES",
    "IMPORTANT": "IMPORTANT",
    "INBOX": "INBOX",
    "Saved Emails/Receipts": "Label_11",
    "Saved Emails/Newsletters": "Label_390122604015354833",
    "To Delete": "Label_6230125154949725760",
    "UNREAD": "UNREAD",
}
label_mapping = {
    EmailDestination.NEWSLETTER: "Saved Emails/Newsletters",
    EmailDestination.BUSINESS_TRANSACTION: "Saved Emails/Receipts",
    EmailDestination.ARCHIVE: None,
    EmailDestination.DELETE: "To Delete",
    EmailDestination.INBOX: "INBOX",
}
label_mapping = {k: label_ids.get(v) for k, v in label_mapping.items()}

inbox_manager = InboxManager(Model.HAIKU, email_summarizer, gmail_client, label_mapping)

inbox_manager_shell = InboxManagerShell(inbox_manager, unread_threads)
executed_actions = inbox_manager_shell.run()

# Experimentation

In [ ]:
from npc.apis.gmail_client import ezgmail

all_threads = ezgmail.search('')
all_labels = set()

# First get all label IDs
for thread in all_threads:
    for message in thread.messages:
        all_labels.update(message.messageObj['labelIds'])

# Get the mapping of label IDs to names
labels_response = ezgmail.SERVICE_GMAIL.users().labels().list(userId='me').execute()
label_dict = {label['id']: label['name'] for label in labels_response['labels']}

# Print labels using their names
print("Available labels:")
for label_id in sorted(all_labels):
    if label_id in label_dict:
        print(label_id, label_dict[label_id])